In [ ]:
import numpy as np
import pandas as pd
import random

def generate_synthetic_data(num_records=3000, random_state=42):
    """
    Genera un DataFrame sintético con registros de eventos de la app.
    Incluye: etiqueta de queja, mensaje, datos socioeconómicos, antecedentes laborales,
    información de viajes y una variable objetivo simulada para la alta probabilidad de retiro.
    """
    np.random.seed(random_state)
    random.seed(random_state)
    
    # Definición de categorías para la etiqueta
    tags = ['finanzas', 'recursos humanos', 'operaciones']
    
    data = []
    for i in range(num_records):
        tag = random.choice(tags)
        message = f"Mensaje sobre {tag} - situación {random.randint(1, 100)}"
        
        # Datos socioeconómicos
        age = np.random.randint(20, 60)
        salary = np.random.randint(2000, 8000)
        education = random.choice(['primaria', 'secundaria', 'universidad', 'postgrado'])
        marital_status = random.choice(['soltero', 'casado', 'divorciado', 'viudo'])
        
        # Antecedentes laborales
        years_experience = np.random.randint(0, max(1, age - 18))
        num_prev_employers = np.random.randint(0, 5)
        
        # Información de viajes
        trip_duration = np.random.randint(10, 180)  # duración en minutos
        distance_traveled = np.random.uniform(1, 100)  # distancia en km
        risk_zone = random.choice(['alta', 'media', 'baja'])
        
        # Simulación de la variable objetivo: alta probabilidad de retiro
        # Se asigna mayor probabilidad si el salario es bajo, la zona de riesgo es alta, etc.
        leave_prob = 0
        if salary < 4000:
            leave_prob += 0.3
        if risk_zone == 'alta':
            leave_prob += 0.3
        if years_experience < 3:
            leave_prob += 0.2
        if tag == 'operaciones':
            leave_prob += 0.1
        # Añadir ruido aleatorio
        leave_prob = min(max(leave_prob + np.random.uniform(-0.1, 0.1), 0), 1)
        high_prob_leave = 1 if leave_prob > 0.5 else 0
        
        record = {
            'tag': tag,
            'message': message,
            'age': age,
            'salary': salary,
            'education': education,
            'marital_status': marital_status,
            'years_experience': years_experience,
            'num_prev_employers': num_prev_employers,
            'trip_duration': trip_duration,
            'distance_traveled': distance_traveled,
            'risk_zone': risk_zone,
            'high_prob_leave': high_prob_leave
        }
        data.append(record)
    
    df = pd.DataFrame(data)
    return df

# Generar y guardar la base de datos sintética
df = generate_synthetic_data(3000)

In [ ]:

# 1. salario por km (evitamos división por cero)
df["salario_por_km"] = df["salary"] / (df["distance_traveled"] + 1e-3)

# 2. experiencia normalizada
df["experiencia_normalizada"] = df["years_experience"] / df["age"]

# 3. salario por duración
df["salario_por_duracion"] = df["salary"] / (df["trip_duration"] + 1e-3)

# 4. is_solo_provider: casado con salario bajo (<4000 como umbral tentativo)
df["is_solo_provider"] = ((df["marital_status"] == "casado") & (df["salary"] < 4000)).astype(int)

# 5. zona_alta_y_sueldo_bajo: riesgo zona alta y salario bajo
df["zona_alta_y_sueldo_bajo"] = ((df["risk_zone"] == "alta") & (df["salary"] < 4000)).astype(int)

df.head()

,driver_id,tag,age,salary,education,marital_status,years_experience,num_prev_employers,trip_duration,distance_traveled,risk_zone,high_prob_leave,message,salario_por_km,experiencia_normalizada,salario_por_duracion,is_solo_provider,zona_alta_y_sueldo_bajo
0,3571958,operaciones,48,7390,primaria,soltero,10,4,30,16.445845,baja,0,"Hola, me gustaría que revisaran mi salario, ya...",449.326288,0.208333,246.325122,0,0
1,3880410,recursos humanos,30,5444,secundaria,casado,3,2,159,6.584746,alta,0,Con un salario de 5444 pesos y viajes de casi ...,826.633723,0.100000,34.238778,0,0
2,3777069,operaciones,57,4433,primaria,soltero,20,0,67,52.950887,baja,0,Me siento frustrado porque después de 20 años ...,83.717508,0.350877,66.163192,0,0
3,3842752,recursos humanos,46,3082,primaria,soltero,9,3,24,46.150928,alta,1,Me siento frustrado porque con 9 años de exper...,66.779441,0.195652,128.411316,0,1
4,3831490,finanzas,22,5556,secundaria,soltero,2,4,82,7.440108,baja,0,"Con el salario de 5556 pesos al mes, apenas pu...",746.663027,0.090909,67.755271,0,0


In [ ]:
df.to_csv(r"C:\Users\igaby\OneDrive\Documents\MEGA\ds_projects\traxion\bronze\driver_data.csv", index=False)